# Gait Report

This notebook generates a gait emg and kinematics report from Cionic sensor data.

The gait report is constructed in the following steps

1. Load the npz for the desired collection
2. Configure kinematics
3. Calculate limb and joint angles
4. Calculate normalized EMG
5. Calculate segments

Once the calculations are complete we can visualize the gait report

1. Knee joint kinematics
2. Hip joint kinematics
3. EMG analysis
4. Gait animation

**Normative kinematics data** were obtained from Fukuchi et al. 2018 ([paper here](https://peerj.com/articles/4640/)) ([data here](https://figshare.com/articles/dataset/A_public_data_set_of_overground_and_treadmill_walking_kinematics_and_kinetics_of_healthy_individuals/5722711/4)). The normative data consists of gait cycle data from 42 individuals of all ages (21–84) and genders (female, male) walking on a treadmill at comfortable speeds (8 speeds, numbered "T1–8"). Kinematics data of all speeds (T1–8) are available on the server but the natural speed (T5) is plotted in the current Gait Report. Normative data was cleaned up using the authors' code in MATLAB, and exported as CSV for import into Jupyter notebook. Mean ± standard deviation of study subjects are plotted in the Gait Report. 


**Normative EMG data** were obtained from Lencioni et al. 2019 ([paper here](http://www.nature.com/articles/s41597-019-0323-z)) ([data here](https://figshare.com/collections/Human_kinematic_kinetic_and_EMG_data_during_level_walking_toe_heel-walking_stairs_ascending_descending/4494755)). This dataset consists of sEMG data on 8 muscles recorded from 50 healthy subjects, both genders (25 males and 25 females) with wide age range (6—72 y). Body mass: 18.2—110 kg, body height: 116.6–187.5 cm. Subjects walked barefoot overground at different walking speeds (normal, increasing and decreasing speeds). Foot strikes on ground force plate was used to segment steps. Original dataset was provided in MATLAB. Light processing was first done in MATLAB (output_norm_emg_data.m MATLAB script, and saved processed data in cde/analysis/norm_data/emg_norm_data/from_lencioni_et_al/processed). The [process_norm_emg_data.ipynb](https://github.com/cionicwear/cde/blob/db2dd5e17a5baecd20dcecb6e3d3a179180c042c/analysis/norm_data/emg_norm_data/process_norm_emg_data.ipynb) notebook converts MATLAB structure to Python dictionary that is ready to be imported into this Gait Report notebook (as a pickle file). 

Detailed information on normative kinematics and EMG data processing is available [here](https://docs.google.com/presentation/d/1DhJpgju5Pdb3B_TG-LunMztQ9c6JiQIsCQWjPtYdasU/edit?usp=sharing). 

**Note:** For left and right pelvic_tilt only, added 7° to Cionic data by default to match normative data kinematics.py (in calculate_joint_angles function).

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import pickle

import dill
import json
import numpy as np
import pandas as pd
from IPython.display import display_html

import cionic
from cionic import tools
import kinematics
from kinematics import EMGAxis, GaitCycleAxis, JointAngleAxis
from kinematics_setup import (
    kinematics_setup,
    signal_peaks,
    signal_troughs,
    zero_crossings,
)

presentation = {}

pd.options.display.max_columns = None
pd.options.display.max_rows = None

# presentation mode
# from kinematics import GaitCycleSimpleAxis as GaitCycleAxis
# from kinematics import JointAngleSimpleAxis as JointAngleAxis
# from kinematics import EMGSimpleAxis as EMGAxis
# presentation = {
#    'style'  : '~/analysis/src/darkmode.mplstyle',
#    'title'  : 'off',
#    'legend' : 'off'
# }

## Load Collection

Download or construct the npz for the collection to be analyzed, and save it to the recordings directory.

As a sanity check print out the **positions**, **labels**, and **stream names**



In [ ]:
# param
npzpath = None
download = None
tokenpath = None
title = "title"
normpath = "norm_data"

time_range = None  # [170.0, 220.0]
label = "steps"
download_zip = False

# for running gait notebook outside of runner
# set npzpath
# study = "gait-develop"
# collection_num = 266

# study = "gait"
# collection_num = 593

# npzpath = f'../recordings/cionic/{study}/{collection_num}/cionic_{study}_{collection_num}_seg.npz'
# npzpath = f'../recordings/cionic/{study}/{collection_num}/cionic_{study}_{collection_num}.npz'

In [ ]:
if download:
    cionic.auth(tokenpath=tokenpath)
    cionic.download_npz(npzpath, download)

In [ ]:
# load regs and boundaries from the original npz
onpz = np.load(npzpath)
regs = tools.stream_regs(onpz)
boundaries = cionic.load_boundary_times(onpz)

In [ ]:
# load csv streams as backup
csv_streams = {}
csv_offset_s = -336.4
if download_zip and zipurl:
    cionic.auth(tokenpath=tokenpath)
    cionic.download_zip(zippath, zipurl)
    directory = os.path.dirname(zippath)
    files = cionic.extract_zip(zippath, directory)
    csv_streams = tools.csv_streams(files, directory, skew=(csv_offset_s * 10000))
    metadata = tools.collection_metadata(directory)
    print(title)
    print("--")
    print(metadata['meta'])

In [ ]:
npz = cionic.load_segmented(npzpath)
outpath = os.path.splitext(npzpath)[0] + '.pdf'

if 'position' in npz['segments'].dtype.names:
    print(f"Positions: {set(npz['segments']['position'])}")

if 'label' in npz['segments'].dtype.names:
    print(f"Labels: {set(npz['segments']['label'])}")

if 'stream' in npz['segments'].dtype.names:
    print(f"Stream: {set(npz['segments']['stream'])}")

In [ ]:
# pd.DataFrame(npz['segments'])

## Load normative EMG data from literature
Lencioni et al. 2019 dataset. Options for processed normative EMG data are (1) not offset (normative_emg_data_normmean.p), (2) offset by 10% to the right (normative_emg_data_normmean_shiftx10.p), (3) offset by 15% (normative_emg_data_normmean_shiftx15.p), or (4) offset by 20% (normative_emg_data_normmean_shiftx20.p).

In [ ]:
emgnormpath = f"{normpath}/emg_norm_data/from_Lencioni_et_al"
emgnormfile = (
    'normative_emg_data_norm_mean_rms.p'  # 'normative_emg_data_norm_mean_lp_butter.p'
)
with open(f'{emgnormpath}/{emgnormfile}', 'rb') as fp:
    norm_emg_data_dict = pickle.load(fp)

## Load normative kinematics data from literature
Fukuchi et al. 2018 dataset.

In [ ]:
kinnormpath = f"{normpath}/kinematics_norm_data/from_Fukuchi_et_al"

norm_kin_avgangle = pd.read_csv(
    f'{kinnormpath}/avg_angles_allage_treadmill_comfspeed.csv', header=[0]
)
norm_kin_stdangle = pd.read_csv(
    f'{kinnormpath}/std_angles_allage_treadmill_comfspeed.csv', header=[0]
)

### Extract normative kinematics data: mean and s.d. angles for each limb plane

In [ ]:
norm_kin_data_dict = {}
norm_kin_data_dict['left'] = {}
norm_kin_data_dict['right'] = {}

for limb_plane in norm_kin_avgangle.keys():

    if limb_plane == 'GaitCyclePercent':
        continue  # skip

    if limb_plane[0] == 'L':
        side = 'left'
    elif limb_plane[0] == 'R':
        side = 'right'

    # format keys for dictionary
    limb_plane_new = limb_plane[2:]  # truncate the side indicator
    limb_plane_new = limb_plane_new.replace(' ', '_').replace('/', '_').lower()
    # print(limb_plane_new)

    # extract data
    gcycle = list(norm_kin_avgangle.GaitCyclePercent)[1:]
    ang = norm_kin_avgangle[limb_plane][1:]
    std = norm_kin_stdangle[limb_plane][1:]

    # place data in dictionary
    norm_kin_data_dict[side][limb_plane_new] = {}  # intialize

    # convert data to numbers
    norm_kin_data_dict[side][limb_plane_new]['gait_cycle'] = [int(i) for i in gcycle]
    norm_kin_data_dict[side][limb_plane_new]['avg_angle'] = [float(i) for i in ang]
    norm_kin_data_dict[side][limb_plane_new]['std_angle'] = [float(i) for i in std]

### Match limb angle labels with that of kinematics_setup

In [ ]:
for side in ['right', 'left']:
    # PELVIS (same labels already)
    norm_kin_data_dict[side]['pelvic_tilt'] = norm_kin_data_dict[side].pop(
        'pelvic_tilt'
    )
    norm_kin_data_dict[side]['pelvic_obliquity'] = norm_kin_data_dict[side].pop(
        'pelvic_obliquity'
    )
    norm_kin_data_dict[side]['pelvic_int_rotation'] = norm_kin_data_dict[side].pop(
        'pelvic_rotation'
    )

    # HIPS
    norm_kin_data_dict[side]['hip_flexion'] = norm_kin_data_dict[side].pop(
        'hip_flexion_extension'
    )
    norm_kin_data_dict[side]['hip_adduction'] = norm_kin_data_dict[side].pop(
        'hip_add_abduction'
    )
    norm_kin_data_dict[side]['hip_int_rotation'] = norm_kin_data_dict[side].pop(
        'hip_int_external_rotation'
    )

    # KNEES
    norm_kin_data_dict[side]['knee_flexion'] = norm_kin_data_dict[side].pop(
        'knee_flx_extension'
    )
    norm_kin_data_dict[side]['knee_adduction'] = norm_kin_data_dict[side].pop(
        'knee_add_abduction'
    )
    norm_kin_data_dict[side]['knee_int_rotation'] = norm_kin_data_dict[side].pop(
        'knee_int_external_rotation'
    )

    # ANKLES
    norm_kin_data_dict[side]['dorsi_flexion'] = norm_kin_data_dict[side].pop(
        'ankle_dorsi_plantarflexion'
    )
    norm_kin_data_dict[side]['ankle_inversion'] = norm_kin_data_dict[side].pop(
        'ankle_inv_eversion'
    )
    norm_kin_data_dict[side]['ankle_int_rotation'] = norm_kin_data_dict[side].pop(
        'ankle_add_abduction'
    )

    # FEET
    norm_kin_data_dict[side]['foot_floor_angle'] = norm_kin_data_dict[side].pop(
        'foot_df_plantarflexion'
    )
    norm_kin_data_dict[side]['foot_supination'] = norm_kin_data_dict[side].pop(
        'foot_inv_eversion'
    )
    norm_kin_data_dict[side]['foot_int_rotation'] = norm_kin_data_dict[side].pop(
        'foot_int_external_rotation'
    )

## Configure Kinematics

Using the **Positions** listed above, construct the kinematics_setup dictionary

At the top level are **groups**, typically *left* and *right*  
All signals in a group will be split with the same segment times.

The signals within a group are **angles** and **emgs**  

An **angle** is keyed on the two limb positions that make up the angle,  
and the value is a dictionary keyed on the x, y, or z component of the resultant euler angle.

An **emg** is keyed on the limb position  
and the value is a remmed name which can be common across sides


In [ ]:
segments = pd.DataFrame(npz['segments'])

# filter down to segments labeled steps
if 'label' in segments and label in set(segments['label']):
    segments = segments.query(f'label=="{label}"')
else:
    print(f"label {label} not found using whole collection")

# initialize kinematics
k = kinematics.Kinematics({})

# load streams that match label and positions
for side in kinematics_setup.keys():

    # calculate needed positions from emgs and angles
    positions = set(kinematics_setup[side]['emgs'])
    positions.update(kinematics_setup[side]['pressures'])
    for a in kinematics_setup[side]['angles'].keys():
        positions.add(a[0])
        positions.add(a[1])

    streams = segments.query(f'position in {list(positions)}')
    if "segment_num" in streams.columns:
        streams = streams.sort_values(by=["segment_num"])

    for index, seg in streams.iterrows():
        # to override elapsed_s with time calculated by sample rate
        # k.load_array(side, seg['position'], seg['stream'], npz[seg['path']], regs.get(seg['device'], hz=seg['avg_rate_hz'])

        # to select a specific time range out of a longer recording
        k.load_array(
            side,
            seg['position'],
            seg['stream'],
            npz[seg['path']],
            regs.get(seg['device']),
            time_range=time_range,
        )

        if 'chanpos' in seg:
            k.load_channel_pos(side, seg['position'], seg['stream'], seg['chanpos'])

        if 'calibration' in seg:
            k.load_calibration(side, seg['position'], seg['stream'], seg['calibration'])

In [ ]:
# create deduped segments from label and segment number and then sort by start time
gwlabels = {f"{b['label']} {b['segment']}": b for b in boundaries}
sortlabels = sorted(gwlabels.values(), key=lambda x: x['start_s'])
labels = [b['label'] for b in sortlabels]

if label in labels:
    labels = [label]
else:
    print(f"label {label} not found in {labels} using whole collection")

# load streams that match label and positions
for side in kinematics_setup.keys():

    # calculate needed positions from emgs and angles
    positions = set(kinematics_setup[side]['emgs'])
    positions.update(kinematics_setup[side]['pressures'])
    for a in kinematics_setup[side]['angles'].keys():
        positions.add(a[0])
        positions.add(a[1])

    for position, arr in csv_streams.items():
        if position in positions and k.groups[side].get(position) is None:
            stream = csv_position_streams.get(position, "float")
            k.load_csv_stream(
                side, position, stream, arr, label_range=None, time_range=time_range
            )

## Calculate Normalized EMG

Compute normalized EMG according to **emg_params** 

normalization algorithms
* k.butter_emg 1. convert to uV with 4V reference 2. signal.butter 3. RMS 
* k.butter_emg 1. convert to uV with 4V reference 2. signal.firwin 3. RMS

and remap labels according to **kinematics_setup..['emgs']** For now, we leave `rms_window` and `normalize` commented out. This allows us to RMS and normalize segmented data to match normative data.


In [ ]:
emg_filter_butter = {
    'filter': tools.butter_highpass_filter,
    'filter_order': 5,
    'cutoff_freq': 50,
    # 'rms_window': 301,
    # 'normalize': np.mean,
}

emg_filter_fir = {
    'filter': tools.fir_filter,
    'taps_n': 63,
    'cutoff_freq': 50,
    # 'rms_window': 301,
    # 'normalize': np.mean
}

filter_params = emg_filter_butter

for side in kinematics_setup.keys():
    k.calculate_emgs(side, kinematics_setup[side]['emgs'], filter_params)

## Calculate Normalized Footbed Pressures

Currently just remap labels according to **kinematics_setup..['foot']**


In [ ]:
# stream = "ladc" # for DC based foot pressure
stream = "emg"  # for SI based foot pressure

params = {'norm': np.max}

for side in kinematics_setup.keys():
    k.calculate_pressures(
        side, kinematics_setup[side]['pressures'], stream=stream, pressure_params=params
    )

## Calculate Limb and Joint Angles

Once the kinematics engine is configured,  
we can compute the angles of the limbs  
and the angles of the joints.

This step applies the sensor calibrations  
and then converts the results into euler angles for presentation.


In [ ]:
# default neutral offsets to match literature
# eventually can read these in from capture
neutral_offsets = {
    'left': {
        'pelvic_tilt': 7,
    },
    'right': {
        'pelvic_tilt': 7,
    },
}

# calculate joint angles an group
for side in kinematics_setup.keys():
    k.calculate_joint_angles(
        side, kinematics_setup[side]['angles'], neutral_offsets=neutral_offsets
    )

    positions = set()
    for a in kinematics_setup[side]['angles'].keys():
        positions.add(a[0])
        positions.add(a[1])

    k.calculate_limb_angles(side, list(positions))

## Calculate Splits

There are multiple methods of calculating the kinematics splits.

The current methodology is to use a peak detector on one of the calibrated euler angle streams.

We use [scipy.signal.find_peaks](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html)
passing the parameters of the **peaks** dict in directly.


The **skips entry** is used to skip errant or incomplete windows
Typical usage is to skip the first and last segment


In [ ]:
# tuning params for walking ranges
n_start_remove = 2
n_stop_remove = 1


def get_walking_time_ranges(side):

    if side not in k.groups.keys():
        return None
    shank_key = f"{side[0].lower()}_shank"
    if shank_key not in k.groups[side]:
        return None
    if 'euler' not in k.groups[side][shank_key]:
        return None
    shank_data = k.groups[side][shank_key]["euler"]
    time_ranges = kinematics.get_walking_intervals(
        shank_data, n_start_remove=n_start_remove, n_stop_remove=n_stop_remove
    )
    return time_ranges


for side in kinematics_setup.keys():
    time_ranges = get_walking_time_ranges(side)
    k.time_ranges[side] = time_ranges

In [ ]:
# configure to split on x euler
heel_strike = {
    'left': {
        'split': ('l_shank', 'euler', 'x'),
        'func': signal_peaks,
        'config': {'height': 0, 'distance': 60},
    },
    'right': {
        'split': ('r_shank', 'euler', 'x'),
        'func': signal_peaks,
        'config': {'height': 0, 'distance': 60},
    },
}

toe_off = {
    'left': {
        'split': ('l_shank', 'euler', 'x'),
        'func': signal_troughs,
        'config': {'height': 0, 'distance': 60},
    },
    'right': {
        'split': ('r_shank', 'euler', 'x'),
        'func': signal_troughs,
        'config': {'height': 0, 'distance': 60},
    },
}

max_knee = {
    'left': {
        'split': ('knee_flexion', 'angle', 'degrees'),
        'func': signal_peaks,
        'config': {'height': 0, 'distance': 60},
    },
    'right': {
        'split': ('knee_flexion', 'angle', 'degrees'),
        'func': signal_peaks,
        'config': {'height': 0, 'distance': 60},
    },
}

mid_swing = {
    'left': {
        'split': ('l_shank', 'euler', 'x'),
        'func': zero_crossings,
        'config': {'sign': '+'},
    },
    'right': {
        'split': ('r_shank', 'euler', 'x'),
        'func': zero_crossings,
        'config': {'sign': '+'},
    },
}

mid_stance = {
    'left': {
        'split': ('l_shank', 'euler', 'x'),
        'func': zero_crossings,
        'config': {'sign': '-'},
    },
    'right': {
        'split': ('r_shank', 'euler', 'x'),
        'func': zero_crossings,
        'config': {'sign': '-'},
    },
}

pconf = {
    'heel_strike': heel_strike,
    'toe_off': toe_off,
    'max_knee': max_knee,
    'mid_swing': mid_swing,
    'mid_stance': mid_stance,
}

k.calculate_splits(pconf, skip_splits_outside_valid_range=True)
k.set_contras('left', 'right', ['heel_strike', 'toe_off'])

k.plot(
    ['left'],
    positions=['l_shank'],
    streams=['euler', 'splits'],
    components=['x', 'heel_strike', 'toe_off'],
    offset=-10,
    width=30,
    height=10,
)
k.plot(
    ['right'],
    positions=['r_shank'],
    streams=['euler', 'splits'],
    components=['x', 'heel_strike', 'toe_off'],
    offset=-10,
    width=30,
    height=10,
)

k.plot(
    ['left'],
    positions=['knee_flexion'],
    streams=['angle', 'splits'],
    components=['degrees', 'max_knee'],
    offset=-10,
    width=30,
    height=10,
)
k.plot(
    ['right'],
    positions=['knee_flexion'],
    streams=['angle', 'splits'],
    components=['degrees', 'max_knee'],
    offset=-10,
    width=30,
    height=10,
)

k.plot(
    ['left'],
    positions=['l_shank'],
    streams=['euler', 'splits'],
    components=['x', 'mid_swing', 'mid_stance'],
    offset=-10,
    width=30,
    height=10,
)
k.plot(
    ['right'],
    positions=['r_shank'],
    streams=['euler', 'splits'],
    components=['x', 'mid_swing', 'mid_stance'],
    offset=-10,
    width=30,
    height=10,
)


split = 'heel_strike'

## Output processed `Kinematics` object

In [ ]:
pklpath = f'{outpath.split(".pdf")[0]}_kinematics.pkl'
dill.dump(k, open(pklpath, 'wb'))

## Gait Timings

[definitions](https://ouhsc.edu/bserdac/dthompso/web/gait/terms.htm)

Step, stance, and swing of each step in the report

### Stance phase
- Loading response begins with initial contact, the instant the foot contacts the ground, and ends with contralateral toe off, when the opposite extremity leaves the ground.
- Midstance begins with contralateral toe off and ends when the center of gravity is directly over the reference foot.
- Terminal stance begins when the center of gravity is over the supporting foot and ends when the contralateral foot contacts the ground. (35% of gait cycle)
- Preswing begins at contralateral initial contact and ends at toe off, at around 60 percent of the gait cycle.

### Swing phase
- Initial swing begins at toe off and continues until maximum knee flexion (60 degrees) occurs.
- Midswing is the period from maximum knee flexion until the tibia is vertical or perpendicular to the ground.
- Terminal swing begins where the tibia is vertical and ends at initial contact.

In [ ]:
def gait_timings(side, contra):
    df = pd.DataFrame()

    if 'heel_strike' not in k.splits[side]:
        return df

    df['step'] = k.split_times(side, 'heel_strike', 'heel_strike')

    if 'toe_off' not in k.splits[side]:
        return df

    toes = k.split_times(side, 'heel_strike', 'toe_off')
    swings = k.split_times(side, 'toe_off', 'heel_strike')

    flen = min(len(df), len(toes), len(swings))
    df = df[:flen]
    df['toe_off'] = toes[:flen]
    df['swing'] = swings[:flen]

    # markers for mid_swing and mid_stance
    if 'mid_stance' in k.splits[side] and 'mid_swing' in k.splits[side]:
        m_mid_stance = k.split_times(side, 'heel_strike', 'mid_stance')
        m_mid_swing = k.split_times(side, 'heel_strike', 'mid_swing')
        flen = min(len(df), len(m_mid_stance), len(m_mid_swing))
        df = df[:flen]
        df['m_mid_stance'] = m_mid_stance[:flen]
        df['m_mid_swing'] = m_mid_swing[:flen]

    # swing breakdown
    if 'max_knee' in k.splits[side] and 'mid_swing' in k.splits[side]:
        init_swings = k.split_times(side, 'toe_off', 'max_knee')
        mid_swings = k.split_times(side, 'max_knee', 'mid_swing')
        term_swings = k.split_times(side, 'mid_swing', 'heel_strike')

        flen = min(len(df), len(init_swings), len(mid_swings), len(term_swings))
        df = df[:flen]
        df['init_swing'] = init_swings[:flen]
        df['mid_swing'] = mid_swings[:flen]
        df['term_swing'] = term_swings[:flen]

    # stance breakdown
    if (
        'contra_toe_off' in k.splits[side]
        and 'contra_heel_strike' in k.splits[side]
        and 'mid_stance' in k.splits[side]
    ):
        # gait events
        loading = k.split_times(side, 'heel_strike', 'contra_toe_off')
        mid_stance = k.split_times(side, 'contra_toe_off', 'mid_stance')
        term_stance = k.split_times(side, 'mid_stance', 'contra_heel_strike')
        pre_swing = k.split_times(side, 'contra_heel_strike', 'toe_off')

        # markers
        m_contra_toe = k.split_times(side, 'heel_strike', 'contra_toe_off')
        m_contra_heel = k.split_times(side, 'heel_strike', 'contra_heel_strike')

        flen = min(
            len(df),
            len(loading),
            len(mid_stance),
            len(term_stance),
            len(pre_swing),
            len(m_contra_toe),
            len(m_contra_heel),
        )
        df = df[:flen]

        # gait phases
        df['loading'] = loading[:flen]
        df['mid_stance'] = mid_stance[:flen]
        df['term_stance'] = term_stance[:flen]
        df['pre_swing'] = pre_swing[:flen]

        # markers
        df['m_contra_toe'] = m_contra_toe[:flen]
        df['m_contra_heel'] = m_contra_heel[:flen]

    return df


timings = {
    'left': {
        'times': gait_timings('left', 'right'),
        'means': {},
        'stdvs': {},
        'percs': {},
    },
    'right': {
        'times': gait_timings('right', 'left'),
        'means': {},
        'stdvs': {},
        'percs': {},
    },
}

markup = '''
<table><tr>
'''

for side in ['left', 'right']:

    df = timings[side]['times']
    if not 'step' in df:
        continue

    mean_step = np.mean(df['step'])
    for name, column in df.items():
        col_mean = np.mean(column)
        timings[side]['means'][name] = np.mean(column)
        timings[side]['stdvs'][name] = np.std(column)
        timings[side]['percs'][name] = np.mean(column) / mean_step


for side in ['left', 'right']:
    df = timings[side]['times']
    df_styler = df.style.set_table_attributes('style="display:inline;"')
    means = timings[side]['means']
    percs = timings[side]['percs']
    stdvs = timings[side]['stdvs']

    markup += f"""
<td style="vertical-align:top;width:400px;overflow-x:scroll">  
<b>{side}</b>
<pre>
     stride  {means.get('step', 0):.2f} ±{stdvs.get('step', 0):.3f}
     stance  {means.get('toe_off', 0):.2f} {percs.get('toe_off', 0)*100:.2f}%
      swing  {means.get('swing', 0):.2f} {percs.get('swing', 0)*100:.2f}%

 init_swing  {means.get('init_swing', 0):.2f} {percs.get('init_swing', 0)*100:.2f}%
  mid_swing  {means.get('mid_swing', 0):.2f} {percs.get('mid_swing', 0)*100:.2f}%
 term_swing  {means.get('term_swing', 0):.2f} {percs.get('term_swing', 0)*100:.2f}%

    loading  {means.get('loading', 0):.2f} {percs.get('loading', 0)*100:.2f}%
 mid_stance  {means.get('mid_stance', 0):.2f} {percs.get('mid_stance', 0)*100:.2f}%
term_stance  {means.get('term_stance', 0):.2f} {percs.get('term_stance', 0)*100:.2f}%
  pre_swing  {means.get('pre_swing', 0):.2f} {percs.get('pre_swing', 0)*100:.2f}%
</pre>
"""
    if False:
        markup += df_styler._repr_html_()

    markup += "</td>"

markup += '</tr></table>'
display_html(markup, raw=True)

In [ ]:
# average measurements from CSU gazelle study
# for greater accuracy replace with own metrics
right_thigh_cm = left_thigh_cm = 43
right_shank_cm = left_shank_cm = 49
stride_cycles = {}

if left_cycles := k.gait_cycles('left', 'l_thigh', 'l_shank'):
    k.gait_triangles(left_cycles, right_thigh_cm, right_shank_cm)
    stride_cycles['left'] = pd.DataFrame(left_cycles)

if right_cycles := k.gait_cycles('right', 'r_thigh', 'r_shank'):
    k.gait_triangles(right_cycles, left_thigh_cm, left_shank_cm)
    stride_cycles['right'] = pd.DataFrame(right_cycles)

markup = '<table><tr>'

for side in ['left', 'right']:
    sdf = stride_cycles.get(side)
    stride_cm = (0.0, 0.0)
    stride_s = (0.0, 0.0)
    gait_speed = (0.0, 0.0)

    if sdf is not None:
        k.stride_len_predict(sdf, "stride_cm")
        sdf["gait_speed"] = sdf["stride_cm"] / sdf["cycle_s"]
        stride_cm = (np.mean(sdf["stride_cm"]), np.std(sdf["stride_cm"]))
        stride_s = (np.mean(sdf["cycle_s"]), np.std(sdf["cycle_s"]))
        gait_speed = (np.mean(sdf["gait_speed"]), np.std(sdf["gait_speed"]))

    markup += f"""
    <td style="vertical-align:top;width:400px;overflow-x:scroll">  
    <b>{side}</b>
    <pre>
stride_len  {stride_cm[0]:10.1f}cm    ±{stride_cm[1]:2.1f}
  stride_s  {stride_s[0]:11.2f}s    ±{stride_s[1]:2.1f}
  gait_cms  {gait_speed[0]:8.1f}cm/s    ±{gait_speed[1]:2.1f}
    </pre>
    """

markup += '</tr></table>'
display_html(markup, raw=True)

## Gait Report

We plot 3 elements of the gait report for each group

1. knee joint kinematics
2. hip joint kinematics
3. full leg emg



In [ ]:
k.save_open(outpath)
computed = set(list(k.groups['left'].keys()) + list(k.groups['right'].keys()))

# create a markers array for the
mark_colors = ["purple", "black", "brown", "grey", "blue"]
markers = {'left': [], 'right': []}

for side in markers.keys():
    for i, mark in enumerate(
        ['m_contra_toe', 'm_contra_heel', 'm_mid_stance', 'm_mid_swing', 'toe_off']
    ):
        if mark in timings[side]['percs']:
            markers[side].append(
                {
                    'x': timings[side]['percs'][mark],
                    #'stdev' : timings[side]['stdvs'][mark],
                    'label': mark,
                    'color': mark_colors[i],
                }
            )

In [ ]:
pelvic_angles = ['pelvic_tilt', 'pelvic_obliquity', 'pelvic_int_rotation']

if set(pelvic_angles).intersection(computed):
    k.save_text('Pelvis Kinematics', width=20, height=10)
    k.plot_splits(
        split,
        norm_kin_data=norm_kin_data_dict,
        groups=['left', 'right'],
        positions=pelvic_angles,
        streams='angle',
        component='degrees',
        width=20,
        sheight=8,
        xaxis=GaitCycleAxis(),
        yaxis=JointAngleAxis(-30, 30),
        markers=markers,
        presentation=presentation,
    )

In [ ]:
hip_angles = ['hip_flexion', 'hip_adduction', 'hip_int_rotation']

if set(hip_angles).intersection(computed):
    k.save_text('Hip Joint Kinematics', width=20, height=10)
    k.plot_splits(
        split,
        norm_kin_data=norm_kin_data_dict,
        groups=['left', 'right'],
        positions=hip_angles,
        streams='angle',
        component='degrees',
        width=20,
        sheight=8,
        xaxis=GaitCycleAxis(),
        yaxis=JointAngleAxis(-30, 60),
        markers=markers,
        presentation=presentation,
    )

In [ ]:
knee_angles = ['knee_flexion', 'knee_adduction', 'knee_int_rotation']

if set(knee_angles).intersection(computed):
    k.save_text('Knee Joint Kinematics', width=20, height=10)
    k.plot_splits(
        split,
        norm_kin_data=norm_kin_data_dict,
        groups=['left', 'right'],
        positions=knee_angles,
        streams='angle',
        component='degrees',
        width=20,
        sheight=8,
        xaxis=GaitCycleAxis(),
        yaxis=JointAngleAxis(-50, 90),
        markers=markers,
        presentation=presentation,
    )

In [ ]:
ankle_angles = ['dorsi_flexion', 'ankle_inversion', 'ankle_int_rotation']

if set(ankle_angles).intersection(computed):
    k.save_text('Ankle Joint Kinematics', width=20, height=10)
    k.plot_splits(
        split,
        norm_kin_data=norm_kin_data_dict,
        groups=['left', 'right'],
        positions=ankle_angles,
        streams='angle',
        component='degrees',
        width=20,
        sheight=8,
        xaxis=GaitCycleAxis(),
        yaxis=JointAngleAxis(-70, 70),
        markers=markers,
        presentation=presentation,
    )

In [ ]:
thigh_angles = ['thigh_floor_angle', 'thigh_adduction']

if set(thigh_angles).intersection(computed):
    k.save_text('Thigh Kinematics', width=20, height=10)
    k.plot_splits(
        split,
        norm_kin_data=None,
        groups=['left', 'right'],
        positions=thigh_angles,
        streams='angle',
        component='degrees',
        width=20,
        sheight=8,
        xaxis=GaitCycleAxis(),
        yaxis=JointAngleAxis(-80, 30),
        markers=markers,
        presentation=presentation,
    )

In [ ]:
shank_angles = ['shank_floor_angle', 'shank_adduction']

if set(shank_angles).intersection(computed):
    k.save_text('Shank Kinematics', width=20, height=10)
    k.plot_splits(
        split,
        norm_kin_data=None,
        groups=['left', 'right'],
        positions=shank_angles,
        streams='angle',
        component='degrees',
        width=20,
        sheight=8,
        xaxis=GaitCycleAxis(),
        yaxis=JointAngleAxis(-80, 30),
        markers=markers,
        presentation=presentation,
    )

In [ ]:
foot_angles = ['foot_floor_angle', 'foot_supination']

if set(foot_angles).intersection(computed):
    k.save_text('Foot Kinematics', width=20, height=10)
    k.plot_splits(
        split,
        norm_kin_data=norm_kin_data_dict,
        groups=['left', 'right'],
        positions=foot_angles,
        streams='angle',
        component='degrees',
        width=20,
        sheight=8,
        xaxis=GaitCycleAxis(),
        yaxis=JointAngleAxis(-80, 30),
        markers=markers,
        presentation=presentation,
    )

In [ ]:
# rms_emg_after_seg = True: RMS and normalize segmented data before plotting. Flip to False to disable.
emg_config = {
    'left': {
        'l_emg': [
            pos
            for pos in sorted(list(k.groups['left']['l_emg'].keys()))
            if pos not in ["emg", "adcf"]
        ]
    },
    'right': {
        'r_emg': [
            pos
            for pos in sorted(list(k.groups['right']['r_emg'].keys()))
            if pos not in ["emg", "adcf"]
        ]
    },
}
k.save_text("EMG", width=20, height=10)
rms_emg_after_seg = {
    'action': True,
    'rms_window': 301,
    'normalize': np.mean,
}
k.plot_splits(
    split,
    norm_emg_data=norm_emg_data_dict,
    config=emg_config,
    component='emg',
    rms_emg_after_seg=rms_emg_after_seg,
    width=20,
    sheight=5,
    xaxis=GaitCycleAxis(),
    yaxis=EMGAxis(0, 5),
    markers=markers,
    presentation=presentation,
)

In [ ]:
# rms_emg_after_seg = True: RMS and normalize segmented data before plotting. Flip to False to disable.
lower_config = {
    'left': {
        'l_shank_emg': [
            pos
            for pos in sorted(list(k.groups['left']['l_shank_emg'].keys()))
            if pos not in ["emg", "adcf"]
        ]
    },
    'right': {
        'r_shank_emg': [
            pos
            for pos in sorted(list(k.groups['right']['r_shank_emg'].keys()))
            if pos not in ["emg", "adcf"]
        ]
    },
}
k.save_text("Lower Leg EMG", width=20, height=10)
rms_emg_after_seg = {
    'action': True,
    'rms_window': 301,
    'normalize': np.mean,
}
k.plot_splits(
    split,
    norm_emg_data=norm_emg_data_dict,
    config=lower_config,
    component='emg',
    rms_emg_after_seg=rms_emg_after_seg,
    width=20,
    sheight=5,
    xaxis=GaitCycleAxis(),
    yaxis=EMGAxis(0, 5),
    markers=markers,
    presentation=presentation,
)

In [ ]:
# rms_emg_after_seg = True: RMS and normalize segmented data before plotting. Flip to False to disable.
upper_config = {
    'left': {
        'l_thigh_emg': [
            pos
            for pos in sorted(list(k.groups['left']['l_thigh_emg'].keys()))
            if pos not in ["emg", "adcf"]
        ]
    },
    'right': {
        'r_thigh_emg': [
            pos
            for pos in sorted(list(k.groups['right']['r_thigh_emg'].keys()))
            if pos not in ["emg", "adcf"]
        ]
    },
}
rms_emg_after_seg = {
    'action': True,
    'rms_window': 301,
    'normalize': np.mean,
}
k.save_text("Upper Leg EMG", width=20, height=10)
k.plot_splits(
    split,
    norm_emg_data=norm_emg_data_dict,
    config=upper_config,
    component='emg',
    rms_emg_after_seg=rms_emg_after_seg,
    width=20,
    sheight=5,
    xaxis=GaitCycleAxis(),
    yaxis=EMGAxis(0, 5),
    markers=markers,
    presentation=presentation,
)

In [ ]:
pressure_config = {
    'left': {
        'l_footbed': [
            pos
            for pos in sorted(list(k.groups['left']['l_footbed'].keys()))
            if pos not in ["emg", "adcf"]
        ]
    },
    'right': {
        'r_footbed': [
            pos
            for pos in sorted(list(k.groups['right']['r_footbed'].keys()))
            if pos not in ["emg", "adcf"]
        ]
    },
}
k.save_text("Foot Pressure", width=20, height=10)
k.plot_splits(
    split,
    config=pressure_config,
    component='mv',
    width=20,
    sheight=5,
    xaxis=GaitCycleAxis(),
    markers=markers,
    presentation=presentation,
)

In [ ]:
k.save_close()

## Export

export data to csv for import into other programs

In [ ]:
# normalize each step cycle to N points
points = 100

# emg export
#
for side in ['left', 'right']:
    for emg in ['r_shank_emg', 'r_thigh_emg', 'l_shank_emg', 'l_thigh_emg']:
        for chan in k.groups[side][emg].keys():
            splits = k.export_split(split, side, emg, chan, 'emg', points)
            if splits:
                csvpath = f"{os.path.splitext(npzpath)[0]}_{side}_{emg}_{chan}.csv"
                print(f"saving to {csvpath}")
                df = pd.DataFrame(splits)
                df.columns = ['start', 'duration'] + list(range(0, points))
                df.to_csv(csvpath, index=False)

# imu export
#
for side in ['left', 'right']:
    for angle in [
        'hip_flexion',
        'hip_adduction',
        'hip_int_rotation',
        'knee_flexion',
        'knee_adduction',
        'knee_int_rotation',
        'dorsi_flexion',
        'ankle_inversion',
        'ankle_int_rotation',
    ]:
        splits = k.export_split(split, side, angle, 'angle', 'degrees', points)
        if splits:
            outpath = f"{os.path.splitext(npzpath)[0]}_{side}_{angle}.csv"
            print(f"saving to {outpath}")
            df = pd.DataFrame(splits)
            df.columns = ['start', 'duration'] + list(range(0, points))
            df.to_csv(outpath, index=False)

# animation export
#
anim = pd.DataFrame(npz['segments'])
rotations = anim.query(f'stream=="fquat"')
data = tools.join_segments(npz, rotations)
outpath = f"{os.path.splitext(npzpath)[0]}_animation.json"
for dk in data.keys():
    del [data[dk]['dtype']]
with open(outpath, "w") as fp:
    fp.write(json.dumps(data))